## JWT (JSON Web Token) para APIs  
Muito usado em Flask, FastAPI e outros frameworks para APIs REST:

Fluxo:

Usuário faz login → servidor gera um token JWT.

Token é enviado pelo cliente em cada requisição (header Authorization).

Servidor valida assinatura e permissões antes de responder.

Vantagens:

Stateless (não precisa manter sessão no servidor).

Escalável para microsserviços.

Bibliotecas: PyJWT, flask-jwt-extended, fastapi-jwt-auth.

In [1]:
%pip install fastapi

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 26.8 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: C:\Users\belch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
%pip install jwt

   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 20.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: C:\Users\belch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer
import jwt
from datetime import datetime, timedelta

# Configurações
SECRET_KEY = "chave_super_secreta"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Função para criar token
def criar_token(dados: dict):
    to_encode = dados.copy()
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

# Endpoint para login (gera token)
@app.post("/token")
def login(username: str, password: str):
    # Aqui você validaria o usuário no banco
    if username != "admin" or password != "123":
        raise HTTPException(status_code=401, detail="Credenciais inválidas")
    token = criar_token({"sub": username})
    return {"access_token": token, "token_type": "bearer"}

# Função para validar token
def validar_token(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=401, detail="Token expirado")
    except jwt.InvalidTokenError:
        raise HTTPException(status_code=401, detail="Token inválido")

# Rota protegida
@app.get("/protegido")
def protegido(usuario: dict = Depends(validar_token)):
    return {"mensagem": f"Bem-vindo, {usuario['sub']}!"}


## OAuth 2.0 e OpenID Connect
Para autenticação via provedores externos (Google, GitHub, Facebook, etc.):

OAuth 2.0: delega autenticação a outro serviço.

OpenID Connect: extensão do OAuth para obter informações do usuário autenticado.

Bibliotecas: authlib, python-social-auth.

📌 Ideal para: quando não quer gerenciar credenciais diretamente.

In [2]:
%pip install requests_oauthlib

  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: C:\Users\belch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from requests_oauthlib import OAuth2Session

# Configurações do cliente (obtidas no console do provedor OAuth)
CLIENT_ID = "sua_client_id"
CLIENT_SECRET = "sua_client_secret"
AUTHORIZATION_BASE_URL = "https://github.com/login/oauth/authorize"
TOKEN_URL = "https://github.com/login/oauth/access_token"
REDIRECT_URI = "http://localhost:5000/callback"

# Escopos de acesso
SCOPE = ["read:user", "user:email"]

# 1️⃣ Criar sessão OAuth2
oauth = OAuth2Session(CLIENT_ID, redirect_uri=REDIRECT_URI, scope=SCOPE)

# 2️⃣ Obter URL de autorização e redirecionar o usuário
authorization_url, state = oauth.authorization_url(AUTHORIZATION_BASE_URL)
print("Acesse este link para autorizar:", authorization_url)

# 3️⃣ Após o login, o provedor redireciona para REDIRECT_URI com um código
redirect_response = input("Cole aqui a URL de redirecionamento: ")

# 4️⃣ Trocar o código pelo token de acesso
token = oauth.fetch_token(
    TOKEN_URL,
    authorization_response=redirect_response,
    client_secret=CLIENT_SECRET
)

print("Token de acesso:", token)

# 5️⃣ Usar o token para acessar recursos protegidos
resp = oauth.get("https://api.github.com/user")
print("Dados do usuário:", resp.json())


## Controle de acesso baseado em funções (RBAC) e atributos (ABAC)
RBAC: permissões atribuídas a funções (roles), e funções atribuídas a usuários.

ABAC: permissões baseadas em atributos do usuário, recurso ou contexto.

Pode ser implementado com bibliotecas como casbin ou usando o próprio sistema de permissões do Django.

## Principais falhas de segurança conhecidas
1. Validação incorreta do redirect_uri  
Permitir wildcards ou correspondências parciais no redirect_uri pode permitir que um invasor redirecione o token para um domínio malicioso.  
Mitigação: exigir correspondência exata e pré-cadastro de URLs confiáveis.

2. Ausência ou uso incorreto do parâmetro state  
O state serve como proteção contra CSRF no fluxo OAuth.
Se não for usado ou validado, um atacante pode iniciar um fluxo OAuth e associar a conta da vítima ao seu próprio perfil.  
Mitigação: sempre gerar e validar state de forma imprevisível.

3. Exposição de tokens no front-end  
Fluxos inseguros (como o Implicit Flow) podem expor tokens no fragmento da URL, facilitando o roubo via histórico do navegador ou logs.  
Mitigação: preferir o Authorization Code Flow com PKCE.

4. Falta de validação do access_token  
Usar tokens sem verificar se foram emitidos para o client_id correto ou se o escopo corresponde ao esperado pode permitir acesso indevido.  
Mitigação: validar emissor, público, escopo e expiração.

1. Configuração fraca do servidor OAuth  
Falta de HTTPS, chaves de assinatura fracas ou endpoints expostos podem comprometer todo o fluxo.  
Mitigação: usar TLS, rotação de chaves e hardening do servidor.

6. Phishing via páginas de autorização falsas  
Usuários podem ser enganados a autorizar apps falsos que imitam a tela de login do provedor.  
Mitigação: educar usuários e usar branding oficial com verificação de domínio.

7. Reuso de códigos de autorização  
Se um código de autorização puder ser reutilizado, um atacante pode trocá-lo por tokens adicionais.  
Mitigação: invalidar o código imediatamente após o uso.